In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from sklearn import preprocessing, pipeline, ensemble, compose

In [2]:
orig = pd.read_csv('/home/sonia/TabFairGAN/adult/adult.csv')
train = orig
orig.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
ordenc = preprocessing.OrdinalEncoder()
numenc = preprocessing.StandardScaler()
lb = preprocessing.LabelBinarizer()

ords = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
nums = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
labs = ['income']

In [4]:
rfc = ensemble.RandomForestClassifier()

In [5]:
preprocessing_pipeline = compose.ColumnTransformer([
    ("ordinal_preprocessor", ordenc, ords),
    ("numerical_preprocessor", numenc, nums),
])
complete_pipeline = pipeline.Pipeline([
    ("preprocessor", preprocessing_pipeline),
    ("estimator", rfc)
])

In [6]:
preprocessed_labels = lb.fit_transform(train[labs])
complete_pipeline.fit(train[ords+nums], preprocessed_labels)

[('ordinal_preprocessor', OrdinalEncoder(), ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']), ('numerical_preprocessor', StandardScaler(), ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'])]


/mnt/data/sonia/miniconda3/envs/llama2/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('ordinal_preprocessor',
                                                  OrdinalEncoder(),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country']),
                                                 ('numerical_preprocessor',
                                                  StandardScaler(),
                                                  ['age', 'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week'])])),
                ('estimator', RandomForestClassifier())])

In [7]:
complete_pipeline.score(orig[ords+nums], preprocessed_labels)

0.971745628762131